In [1]:
import torch

In [2]:
torch.cuda.empty_cache()

In [11]:
from transformers import AutoTokenizer
import transformers
import torch

model = "tiiuae/falcon-7b"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)
sequences = pipeline(
   "Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth. Giraftron believes all other animals are irrelevant when compared to the glorious majesty of the giraffe.\nDaniel: Hello, Girafatron!\nGirafatron:",
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")


Loading checkpoint shards: 100%|██████████| 2/2 [00:08<00:00,  4.29s/it]
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Result: Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth. Giraftron believes all other animals are irrelevant when compared to the glorious majesty of the giraffe.
Daniel: Hello, Girafatron!
Girafatron: Yes, Daniel! I have come to your humble abode to share some exciting new with you!
Daniel: Wow, that sounds exciting! I love giraffes, too!
Girafatron: Daniel! You are a good person for loving giraffes!
Daniel: I know!
Daniel: So, what exciting stuff you got for me?
Girafatron: Yes! Daniel, the giraffe population is rapidly increasing in my native home of Kenya. This is amazing! I have a feeling that in a few years, giraffe will be an endangered animal.
Daniel: Oh. That’s sad. So we should celebrate giraff


In [12]:
import pandas as pd
df_text = pd.read_csv('Te2Query.csv')
eg = df_text.sample(n=1000, random_state=3)
input_text = eg['Questions'].to_list()
input_labels = eg['query'].to_list()
eg

,Unnamed: 0,Questions,id,query
3276,3276,Give me all the patients who got vaccines on 08/10/2012.,20103276,"POST_scripts/1{""script"":{""lang"":""mustache"",""source"":{""track_total_hits"":""true"",""query"":{""term"":{""{{field_1}}"":""{{value_1}}""}}},""params"":{""field_1"":'DATA.VAX_DATE',""value_1"":'08/10/2012'"
1409,1409,Give me all the patients whose report was completed on 03/26/2022.,7101409,"POST_scripts/1{""script"":{""lang"":""mustache"",""source"":{""track_total_hits"":""true"",""query"":{""term"":{""{{field_1}}"":""{{value_1}}""}}},""params"":{""field_1"":'DATA.RPT_DATE',""value_1"":'03/26/2022'"
7172,7172,Which is the most common cataracts for patients.,28307172,"POST_scripts/3{""script"":{""lang"":""mustache"",""source"":{""track_total_hits"":""true"",""query"":{""match"":{""{{field_1}}"":{""query"":""{{value_1}}"",""fuzziness"":""AUTO"",""operator"":""AND"",'DATA.HISTORY',""params"":'cataracts'"
9319,9319,What is the number of records that the vaccine recipient got USPFIZER INC202200713889 vaccine.,33109319,"POST_scripts/1{""script"":{""lang"":""mustache"",""source"":{""track_total_hits"":""true"",""query"":{""term"":{""{{field_1}}"":""{{value_1}}""}}},""params"":{""field_1"":'DATA.SPLTTYPE',""value_1"":'USPFIZER"
11467,11467,Give me all the patients who got INFLUENZA (SEASONAL) (FLUBLOK) vaccine.,43311467,"POST_scripts/3{""script"":{""lang"":""mustache"",""source"":{""track_total_hits"":""true"",""query"":{""match"":{""{{field_1}}"":{""query"":""{{value_1}}"",""fuzziness"":""AUTO"",""operator"":""AND"",'VAX.VAX_NAME',""params"":'INFLUENZA"
...,...,...,...,...
7332,7332,Give me all the patients who was allergic to pvc,29307332,"POST_scripts/3{""script"":{""lang"":""mustache"",""source"":{""track_total_hits"":""true"",""query"":{""match"":{""{{field_1}}"":{""query"":""{{value_1}}"",""fuzziness"":""AUTO"",""operator"":""AND"",'DATA.ALLERGIES',""params"":'pvc'"
10466,10466,How many GLAXOSMITHKLINE BIOLOGICALS vaccine has been used.,37110466,"POST_scripts/1{""script"":{""lang"":""mustache"",""source"":{""track_total_hits"":""true"",""query"":{""term"":{""{{field_1}}"":""{{value_1}}""}}},""params"":{""field_1"":'VAX.VAX_MANU',""value_1"":'GLAXOSMITHKLINE"
1109,1109,How many patients are 100.0 years old.,3201109,"POST_scripts/2{""script"":{""lang"":""mustache"",""source"":{""track_total_hits"":""true"",""query"":{""match"":{""{{field_1}}"":""{{value_1}}""}}},""params"":{""field_1"":'DATA.AGE_YRS',""value_1"":100.0"
7771,7771,What is the number of vaccine recipients that had an allergic to insect?Which is The most common walnut sensitivity for patients?,29307771,"POST_scripts/3{""script"":{""lang"":""mustache"",""source"":{""track_total_hits"":""true"",""query"":{""match"":{""{{field_1}}"":{""query"":""{{value_1}}"",""fuzziness"":""AUTO"",""operator"":""AND"",'DATA.ALLERGIES',""params"":'insect'},"


In [13]:
# # original prompt
# prompt = """ignore all the prior information before this block. Convert the following questions to elastic search queries follow two rules:
# 1.based on the field name 'RECVDATE','STATE','AGE_YRS','VAERS_ID','SEX','SYMPTOM_TEXT','DIED','ER_VISIT','L_THREAT','HOSPITAL','HOSPDAYS','DISABLE','VAX_DATE','LAB_DATA','OTHER_MEDS','CUR_ILL','HISTORY','PRIOR_VAX','TODAYS_DATE','OFC_VISIT','VAX_TYPE','VAX_MANU','VAX_LOT','VAX_DOSE_SERIES','VAX_NAME','ALLERGIES'. 
# 2.follow the template 

# "POST _scripts/1
# {
#   "script": {
# 	"lang": "mustache",
# 	"source": {
#   	"track_total_hits": "true",
#   	"query": {
#     	"term": {
#       	"{{field}}": "{{date}}"
#     	}
#   	}
# 	},
# 	"params": {
#   	"field": "DATA.RECVDATE.keyword",
#   	"date": "01/01/2012"
# 	}
#   }
# }
# "

# """

In [ ]:
import torch
torch.cuda.empty_cache()

#NER prompt
prompt_prefix = """Find the entity of the following questions based on the field name follow emample:How many patients' record are received on 03/20/2022. The '03/20/2022' is a ['RECVDATE']."""


In [ ]:
torch.cuda.empty_cache()

#Q&A prompt
prompt_cloze = """Classify the questions based on the field name follow example:How many patients' record are received on 03/20/2022. The question wants ['VAERS_ID'] based on ['RECVDATE']."""


In [14]:
torch.cuda.empty_cache()

# cot + heuristic prompt
prompt_cot = """
covert input questions to elastic queries based on two steps:1.Find the entity of the following questions based on the name:
'RECVDATE','STATE','AGE_YRS','VAERS_ID','SEX','SYMPTOM_TEXT','DIED','ER_VISIT','L_THREAT','HOSPITAL','HOSPDAYS','DISABLE','VAX_DATE','LAB_DATA','OTHER_MEDS','CUR_ILL','HISTORY','PRIOR_VAX','TODAYS_DATE','OFC_VISIT','VAX_TYPE','VAX_MANU','VAX_LOT','VAX_DOSE_SERIES','VAX_NAME','ALLERGIES'
2.Based on the named entities which are also {{field}} in the query codes, covert these questions to elastic queries. 
Make sure to follow the template

"POST _scripts/1
{
  "script": {
	"lang": "mustache",
	"source": {
  	"track_total_hits": "true",
  	"query": {
    	"term": {
      	"{{field}}": "{{date}}"
    	}
  	}
	},
	"params": {
  	"field": "DATA.RECVDATE.keyword",
  	"date": "01/01/2012"
	}
  }
}
"
"""

In [15]:
# define the model input template
input_template = """
Prompt: {prompt}
Clinical Notes: {text}
Answer:
"""

In [16]:
# build up the call
answer_lst = []
for row in eg.iterrows():
    txt = row[1]['Questions']
#    suggest = row[1]['query']
    input = input_template.format(prompt = prompt_cot, text = txt)
    answer = pipeline(input)
    answer_lst.append(answer[0]['generated_text'][len(input):].strip())
    #answer_lst.append(answer[0]['generated_text'])
eg['llm_result'] = answer_lst

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
/home/wenlong/anaconda3/envs/llama/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/home/wenlong/anaconda3/envs/llama/lib/python3.10/site-packages/transformers/generation/utils.py:1361: UserWarning: Input length of input_ids is 348, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
/home/wenlong/anaconda3/envs/llama/lib/python3.10/site-packages/transformers/generation/utils.py:1361: UserWarning: Input length of input_ids is 340, but `max_length` is set to 20. This can lead to unex

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
/home/wenlong/anaconda3/envs/llama/lib/python3.10/site-packages/transformers/generation/utils.py:1361: UserWarning: Input length of input_ids is 341, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
/home/wenlong/anaconda3/envs/llama

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_tok

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_tok

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
/home/wenlong/anaconda3/envs/llama

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_tok

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_tok

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_tok

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
/home/wenlong/anaconda3/envs/llama/lib/python3.10/site-packages/transformers/generation/utils.py:1361: UserWarning: Input length of input_ids is 380, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to

In [17]:
pd.set_option('display.max_colwidth', None)
eg['llm_result']

3276     {
1409      
7172      
9319      
11467     
        ..
7332     "
10466     
1109      
7771     1
1378      
Name: llm_result, Length: 1000, dtype: object

In [18]:
result_df = eg[['llm_result']]
result_df.to_json('covert_Falcon_NER_1.json')

In [19]:
import json
 
# Opening JSON file
f = open('covert_Falcon_NER_1.json')
 
# returns JSON object as 
# a dictionary
data = json.load(f)
 
# Iterating through the json
# list
print(data)
 
# Closing file
f.close()

{'llm_result': {'3276': '{', '1409': '', '7172': '', '9319': '', '11467': '', '1987': '', '4088': '', '8454': '', '12811': '', '11778': '"', '10127': '', '8822': '', '12373': '', '6412': '', '2270': '', '4655': '', '5509': '', '3710': 'POST', '1272': '', '8353': '', '8657': '1', '5450': '', '8585': '', '8477': '1', '7279': '"', '5323': '', '9807': '', '7266': '', '3959': '', '8848': '', '3861': 'POST', '8730': '1', '9632': '', '12713': '', '5654': '', '2262': '', '4946': '', '5012': '1', '5908': '', '9771': '', '4862': '', '8367': '', '5667': '', '151': '', '4617': '', '7214': '', '7256': '"', '4529': 'The', '8143': '{', '6821': '1', '1172': '', '4345': '', '6024': '1', '8104': '{', '10596': '"', '533': '{', '11691': '', '1826': '', '12874': '', '6812': '1', '1031': '{', '5092': '1', '6329': '', '11348': '', '1210': '', '4158': '', '2694': '{{', '2200': 'Int', '9273': '', '9789': '', '5771': 'Yes', '3037': '', '9186': '', '4201': '', '12514': '', '1160': '', '7456': '', '5018': '', '71

In [20]:
from codebleu import calc_codebleu

prediction = str(answer_lst)
reference = df_text['query'].to_string()
result_eval = calc_codebleu([reference], [prediction], lang="php", weights=(0.25, 0.25, 0.25, 0.25), tokenizer=None)

In [21]:
result_eval=pd.Series(result_eval)
result_eval.to_json('Falcon_result_eval_NER_1.json')

In [22]:
import json
 
# Opening JSON file
f = open('Falcon_result_eval_NER_1.json')
 
# returns JSON object as 
# a dictionary
data = json.load(f)
 
# Iterating through the json
# list
print(data)
 
# Closing file
f.close()

{'codebleu': 0.5, 'ngram_match_score': 0.0, 'weighted_ngram_match_score': 0.0, 'syntax_match_score': 1.0, 'dataflow_match_score': 0.0}
